In [2]:
import pandas as pd

# Load labeled dataset of legal clauses
# Assumes 'labeled.csv' has columns: filename, label, text, basis
df_raw = pd.read_csv('./labeled.csv')

# Extract domain from filename (format: '001_<domain>_가공.json')
df_raw['domain'] = df_raw['filename'].apply(lambda x: x.split('_')[1])

# Now df_raw['domain'] contains domain information usable for generating positive/negative pairs


In [5]:
df_raw

,filename,label,text,basis,domain
0,001_개인정보취급방침_가공.json,유리,제2조(개인정보의 처리 및 보유기간) \n① 협회는 법령에 따른 개인정보 보유․이용...,NaN,개인정보취급방침
1,001_결혼정보서비스_가공.json,유리,제3조 (회원가입)\n① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제공...,NaN,결혼정보서비스
2,001_보증_가공.json,유리,제2조(보증금액)\n ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부...,NaN,보증
3,001_사이버몰_가공.json,유리,제3조 (약관 등의 명시와 설명 및 개정)\n① 몰은 이 약관의 내용과 상호 및 대...,NaN,사이버몰
4,001_상해보험_가공.json,유리,제4조(보험금 지급에 관한 세부규정)\n② 제3조(보험금의 지급사유) 제2호에서 장...,NaN,상해보험
...,...,...,...,...,...
7995,620_임대차계약_가공.json,불리,제2조(임대료)\n제2항 임대료 등의 연체시 매월 100분의 10에 해당하는 연체료...,매월 100분의 10은 연120퍼센트의 연체료율이 되어 이자제한법상 연25퍼센트를 ...,임대차계약
7996,621_임대차계약_가공.json,불리,제12조(해약인정 및 보증금 등의 반환)\n제2항 갑은 해약이 확정되면 즉시 을의 ...,점포를 재임대하여 반환하거나 또는 해약일로부터 3개월 이내에 임대보증금을 반환하되 ...,임대차계약
7997,622_임대차계약_가공.json,불리,제17조(임대인의 금지사항)\n제2항 을이 전항 및 제11조와 제14조에 위반 내지...,임대차계약관계에서 임차인이 지켜야 할 사항을 게을리하거나 지키지 않는 경우 임대인은...,임대차계약
7998,623_임대차계약_가공.json,불리,제20조(임대인의 금지조항)\n제1항 본 계약 각 조항의 해석상의 이의가 있는 경우...,해석상 이의가 있는 경우 임대인의 해석에 따르고 계약에 명시되지 않은 사항은 임대인...,임대차계약


In [6]:
# This cell assumes `df_raw` already exists with columns: filename, label, text, basis
import random
import pandas as pd

# Copy raw DataFrame and extract domain from filename
# (format: '001_<domain>_가공.json')
df = df_raw.copy()
df['domain'] = df['filename'].apply(lambda x: x.split('_')[1])

# Prepare lists for sampling
clauses = df['text'].tolist()
domains = df['domain'].tolist()
labels = df['label'].tolist()

pos_clauses = []
neg_clauses = []

for i, (clause, domain, label) in enumerate(zip(clauses, domains, labels)):
    # Positive: sample another clause with same domain AND same label
    same_idxs = [j for j, (d, l) in enumerate(zip(domains, labels)) if d == domain and l == label and j != i]
    if not same_idxs:
        # fallback to same domain only
        same_idxs = [j for j, d in enumerate(domains) if d == domain and j != i]
    pos_clauses.append(clauses[random.choice(same_idxs)])

    # Negative: sample a clause with different domain OR different label
    diff_idxs = [j for j, (d, l) in enumerate(zip(domains, labels)) if d != domain or l != label]
    neg_clauses.append(clauses[random.choice(diff_idxs)])

# Build DataFrame of pairs and save to CSV
df_pairs = pd.DataFrame({
    'clause': clauses,
    'pos_clause': pos_clauses,
    'neg_clause': neg_clauses
})
df_pairs.to_csv('legal_pairs.csv', index=False)
print(f"Saved 'legal_pairs.csv' with {len(df_pairs)} entries.")


Saved 'legal_pairs.csv' with 8000 entries.


In [1]:
import pandas as pd

df_pairs = pd.read_csv('./legal_pairs.csv')

In [2]:
df_pairs

,clause,pos_clause,neg_clause
0,제2조(개인정보의 처리 및 보유기간) \n① 협회는 법령에 따른 개인정보 보유․이용...,제6조(처리하는 개인정보 항목)\n개인정보처리자명 은(는) 다음의 개인정보 항목을 ...,제15조(여행출발 후 계약해지)\n⑥ 제4항에 따라 계약이 해지된 경우 여행사는 계...
1,제3조 (회원가입)\n① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제공...,"제13조 (회원에 대한 통지)\n회원에 대한 통지 및 예고는 서면, 전화, 모사전송...",제7조(00000 이용 제한)\n제3항 이용한도액은 회원의 신용도 및 카드사용실적 ...
2,제2조(보증금액)\n ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부...,제16조 채권자의 협조의무\n③ 000가 조건부융자에 대하여 보증채무를 이행한 후에...,"제30조 계약기간\n본 계약은 계약일로부터 1년간 유효하며, 발효일은 갑이 본 계약..."
3,제3조 (약관 등의 명시와 설명 및 개정)\n① 몰은 이 약관의 내용과 상호 및 대...,"제7조(회원 탈퇴 및 자격 상실 등)\n③ 몰이 회원 자격을 제한․정지 시킨 후, ...",제30조(과오금)\n④ 회사는 이용자 상호간 또는 이용자와 제3자 간에 콘텐츠를 매...
4,제4조(보험금 지급에 관한 세부규정)\n② 제3조(보험금의 지급사유) 제2호에서 장...,제4조(보험금 지급에 관한 세부규정)\n② 제3조(보험금의 지급사유) 제2호에서 장...,제17조(청약의 철회)\n② 제1항에도 불구하고 청약한 날부터 5일이 초과된 계약은...
...,...,...,...
7995,제2조(임대료)\n제2항 임대료 등의 연체시 매월 100분의 10에 해당하는 연체료...,제 4 조 (계약해지)\n임차인은 계약해지의 통지가 있은 시간으로부터 3시간 이내에...,제4항 과다한 위약금 조항 ...
7996,제12조(해약인정 및 보증금 등의 반환)\n제2항 갑은 해약이 확정되면 즉시 을의 ...,"제4조 (보험대리점의 권리의무)\n보험대리점은 보험업 관련 제 법령 및 규정, 금융...",제11조(공시)\n①갑은 매 2일마다 시가로 평가한 순자산가치 및 주식의 1주당 평...
7997,제17조(임대인의 금지사항)\n제2항 을이 전항 및 제11조와 제14조에 위반 내지...,제 4 조(계약위반의 조치\n갑은 을에 대하여 별도로 최고를 요하지 않고 본 계약을...,제9조 비밀엄수 및 영업판매의 양도 등에 관한 사항\n제3항 을 은 이 계약 및 ...
7998,제20조(임대인의 금지조항)\n제1항 본 계약 각 조항의 해석상의 이의가 있는 경우...,제1조\n제2항 을은 중도금 및 종대금을 납부치 않을 시는 본 계약은 최고없이 해제...,제2조(계약의 해제)\n② 을 은 갑 의 귀책사유로 인해 다음 각 호의 어느 하나...


In [4]:
# —– 0. Monkey-patch Dataset and DatasetDict into sentence_transformers —–
from datasets import Dataset as HFDataset, DatasetDict as HFDatasetDict
import sentence_transformers.fit_mixin as fit_mixin

fit_mixin.Dataset = HFDataset
fit_mixin.DatasetDict = HFDatasetDict

# —– 1. Imports & Device —–
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader
import torch

device     = torch.device('cpu')
model_name = 'upskyy/e5-small-korean'

# —– 2. 모델 로딩 —–
word_embedding = models.Transformer(model_name, max_seq_length=256)
pooling        = models.Pooling(word_embedding.get_word_embedding_dimension())
model          = SentenceTransformer(modules=[word_embedding, pooling], device=device)

# —– 3. Contrastive Examples 준비 —–
# df_pairs: DataFrame with columns clause, pos_clause, neg_clause
examples = []
for _, row in df_pairs.iterrows():
    examples.append(InputExample(texts=[row['clause'], row['pos_clause']], label=1.0))
    examples.append(InputExample(texts=[row['clause'], row['neg_clause']], label=0.0))

# —– 4. DataLoader & Loss —–
train_dataloader = DataLoader(examples, shuffle=True, batch_size=9)
train_loss       = losses.ContrastiveLoss(model)

# —– 5. Fine-tuning —–
num_epochs   = 1
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
output_dir   = './legal-kr-sbert-contrastive'

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=output_dir,
    use_amp=False
)

print(f"🚀 Training complete. Model saved to '{output_dir}'.")


c:\Users\user\Documents\JSY\KW\3-1\TextMining\TextMining\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--upskyy--e5-small-korean. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Step,Training Loss
500,0.018500
1000,0.010700
1500,0.009100


🚀 Training complete. Model saved to './legal-kr-sbert-contrastive'.
